In [1]:
!pip install transformers 

In [2]:
!pip install sentencepiece

In [3]:
!pip install sacremoses

  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8a6ec4e7c025e61015bca0528350f79f1376b3f891d9de635e76480ddbc7cff4
  Stored in directory: c:\users\nasser\appdata\local\pip\cache\wheels\12\1c\3d\46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


In [33]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer, AdamW
from torch.utils.data import DataLoader, Dataset
import sacremoses
import sentencepiece


In [36]:
# Load your parallel corpus data into a pandas DataFrame
data = pd.read_csv("https://raw.githubusercontent.com/NK-Z/HAT/main/Claned_data.csv", index_col=0)  # Adapt the file name and structure

# Define your fine-tuning dataset class
class TranslationDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        source_text = self.data.iloc[idx]["English"]
        target_text = self.data.iloc[idx]["Arabic"]

        # Tokenize source and target sentences
        source_tokens = self.tokenizer.encode(source_text, padding="max_length", truncation=True, return_tensors="pt")
        target_tokens = self.tokenizer.encode(target_text, padding="max_length", truncation=True, return_tensors="pt")

        return {
            "input_ids": source_tokens.squeeze(),
            "attention_mask": source_tokens.squeeze().gt(0),  # Create attention mask
            "labels": target_tokens.squeeze(),
        }

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
data

,Arabic,English
0,ﺗﻔﺮﻳﺞ ﺍﻟﻜﺮﻭﺏ ﻓﻰ ﺗﺪﺑﻴﺮ ﺍﻟﺤﺮﻭﺏ,TAFRIJ AL-KURUB FI TADBIR AL-HURUBA Muslim Man...
1,ﻣﻘﺪﻣﺔ,INTRODUCTION
2,ﺑﺴﻢ ﷲ ﺍﻟﺮﺣﻤﻦ ﺍﻟﺮﺣﻴﻢ.. ﻣﺆﻳﺪ ﺍﻹﺳﻼﻡ ﻣﻦ ﺳﻠﻄﺎﻧﻪ ﺍﻟﻨ...,"IN THE NAME OF GOD, THE MERCIFUL, THE COMPASSI..."
3,ﻭﻣﺴﻌﺪ ﺟﺪﻩ ﺍﻟﻌﺎﻟﻰ ﺑﺈﺑﺎﺩﺓ ﺃﻋﺪﺍﺋﻪ ﺍﻟﻄﻐﺎﺓ ﺍﻟﻤﺎﺭﻗﻴﻦ...,And [he is] the cause of his noble sire’s happ...
4,ﻭﺃﺷﻬﺪ ﺃﻥ ﻻ ﺇﻟﻪ ﺇﻻ ﷲ ﻭﺣﺪﻩ ﻻ ﺷﺮﻳﻚ ﻟﻪ، ﺷﻬﺎﺩﺓ ﻳﺘﻮﺍ...,"I declare that there is no god but God alone, ..."
...,...,...
102090,ﺣﺪﺛﻨﺎ ﻋﺒﺪ ﺍﻟﺮﺣﻤﻦ ﺑﻦ ﺇﺑﺮﺍﻫﻴﻢ، ﺣﺪﺛﻨﺎ ﺍﻟﻮﻟﻴﺪ ﺑﻦ ﻣ...,It was narrated from ‘Awf bin Malik Al-Ashja’...
102091,ﺣﺪﺛﻨﺎ ﺃﺑﻮ ﺑﻜﺮ ﺑﻦ ﺃﺑﻲ ﺷﻴﺒﺔ، ﺣﺪﺛﻨﺎ ﺳﻔﻴﺎﻥ ﺑﻦ ﻋﻴﻴﻨ...,It was narrated from Abu Hurairah conveying i...
102092,ﺣﺪﺛﻨﺎ ﺃﺑﻮ ﺑﻜﺮ ﺑﻦ ﺃﺑﻲ ﺷﻴﺒﺔ، ﺣﺪﺛﻨﺎ ﺳﻔﻴﺎﻥ ﺑﻦ ﻋﻴﻴﻨ...,It was narrated from Abu Hurairah that the Me...
102093,ﺣﺪﺛﻨﺎ ﺃﺑﻮ ﺑﻜﺮ ﺑﻦ ﺃﺑﻲ ﺷﻴﺒﺔ، ﺣﺪﺛﻨﺎ ﺃﺳﻮﺩ ﺑﻦ ﻋﺎﻣﺮ،...,It was narrated that ‘Amr bin Taghlib said: ...


In [35]:
# Load pretrained model and tokenizer for Arabic to English translation
model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

In [38]:
# Load pretrained model and tokenizer for Arabic to English translation
model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# Create fine-tuning dataset and dataloader
train_dataset = TranslationDataset(data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss() 

c:\Users\Nasser\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [39]:
num_epochs = 1
best_loss = float('inf')  # Initialize with a high value

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}] - Batch Loss: {loss.item():.4f}')

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Average Loss: {average_loss:.4f}")

    # Save the best model based on training loss
    if average_loss < best_loss:
        best_loss = average_loss
        model.save_pretrained("best_model")

print("Training finished.")

Epoch [1/1] - Batch Loss: 7.6465
Epoch [1/1] - Batch Loss: 7.1134
Epoch [1/1] - Batch Loss: 6.3100
Epoch [1/1] - Batch Loss: 5.5113
Epoch [1/1] - Batch Loss: 5.1814
Epoch [1/1] - Batch Loss: 4.6364
Epoch [1/1] - Batch Loss: 3.9388
Epoch [1/1] - Batch Loss: 3.5361
Epoch [1/1] - Batch Loss: 3.0605
Epoch [1/1] - Batch Loss: 2.1867
Epoch [1/1] - Batch Loss: 2.1195
Epoch [1/1] - Batch Loss: 2.1259
Epoch [1/1] - Batch Loss: 1.6392
Epoch [1/1] - Batch Loss: 1.9548
Epoch [1/1] - Batch Loss: 2.1534
Epoch [1/1] - Batch Loss: 1.9185
Epoch [1/1] - Batch Loss: 1.7422
Epoch [1/1] - Batch Loss: 1.5592
Epoch [1/1] - Batch Loss: 1.2508


KeyboardInterrupt: 

In [5]:
my_model = MarianMTModel.from_pretrained('model')

In [6]:
my_model_2 = MarianMTModel.from_pretrained('Model_2')

In [172]:
# Inference function
def translate_english__to_arabic(input_text):
    input_text = [input_text]
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    translated_ids = my_model_2.generate(input_ids, max_length=len(str(input_text))+20, num_beams=100, early_stopping=True, no_repeat_ngram_size=2).to(device)
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translated_text

In [174]:
# Example usage
input_text = "They fought until they gave up"
translated_text = translate_english__to_arabic(input_text)
print("Input: ", input_text)
print("Translated: ", translated_text)

Input:  They fought until they gave up
Translated:  فقاتلوا حت إلى أَن يَتَرَكَهُم


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save your model
model.save_pretrained('/content/drive/My Drive/best_model_2.h5')